In [20]:
!pip install pubchempy
!pip install rdkit
!pip install chemspipy
!pip install chembl_webresource_client

In [73]:
import pandas as pd
import numpy as np
import pubchempy as pbp
import requests
from rdkit import Chem
from chembl_webresource_client.new_client import new_client
import chemspipy

In [21]:
df = pd.read_csv('6.csv')

In [22]:
df

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/S0022-3093(99)00330-0,7/19/1999,Journal of Non-Crystalline Solids,PHOTOINDUCEDCHANGESINLINEARNONLINEAROPTICALPRO...,As20S60Ge20,0.0,NaN,el_cde_tables,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",2.054,n
1,10.1016/j.fct.2006.05.017,6/7/2006,Food and Chemical Toxicology,GENERATIONFORMALDEHYDEINCIGARETTESOVERVIEWRECE...,Propionaldehyde,0.0,NaN,el_cde_tables,CCC=O,3.74,n
2,10.1016/j.mee.2011.01.031,1/18/2011,Microelectronic Engineering,SINUSOIDALPLASMONICCRYSTALSFORBIODETECTIONSENSORS,PEO,0.0,NaN,el_cde_text,"[['O', 1.0], ['PE', 1.0]]",1.47,refractive index
3,10.1016/j.apradiso.2012.04.026,5/2/2012,Applied Radiation and Isotopes,BENCHMARKINGGEANT4FULLSYSTEMSIMULATIONASSOCIAT...,Aluminum,0.0,NaN,el_cde_tables,[AlH2],4.4,Refractive index
4,10.1016/S0924-4247(02)00264-9,9/16/2002,Sensors and Actuators A: Physical,INFLUENCESMATERIALPROPERTIESCERAMICMICROSTEREO...,Alumina,0.0,NaN,el_mylogic,NaN,1.7,Refractive index
...,...,...,...,...,...,...,...,...,...,...,...
4995,10.1016/j.molliq.2011.07.002,7/22/2011,Journal of Molecular Liquids,UVSPECTRALCHANGESFORAZOCOMPOUNDSINPRESENCEDIFF...,iso-Butanol,0.0,NaN,el_cde_tables,NaN,1.3959,n
4996,10.1016/j.asr.2009.06.015,6/26/2009,Advances in Space Research,DIFFERENTAPPROACHESRAYLEIGHOPTICALDEPTHDETERMI...,CO2,0.0,NaN,el_cde_tables,O=C=O,1.000274585,Refractive index
4997,10.1016/S0022-3093(99)00681-X,2/25/2000,Journal of Non-Crystalline Solids,UVTRANSMISSIONRADIATIONINDUCEDDEFECTSINPHOSPHA...,Zn(PO3)2,0.0,NaN,el_cde_tables,"[['O', 6.0], ['P', 2.0], ['Zn', 1.0]]",1.526,Refractive index
4998,10.1016/j.jct.2016.04.016,4/20/2016,The Journal of Chemical Thermodynamics,ASYMMETRICCRITICALITYIONICSOLUTIONCONTAINING1H...,2-propanol,0.0,632.8 nm,el_mylogic,CC(C)O,1.3884,n


#### Проверить есть ли Nan в колонке 'Name' и normalised_name

In [23]:
if df['Name'].isna().any():
    print('Колонка Name содержит пропущенные значения')
else:
    print('Колонка Name не содержит пропущенных значений')
if df['normalised_name'].isna().any():
    print('Колонка normalised_name содержит пропущенные значения')
else:
    print('Колонка normalised_name не содержит пропущенных значений')

Колонка Name содержит пропущенные значения
Колонка normalised_name содержит пропущенные значения


#### Проверим есть ли строки, в которых одновременно в двух колонках Name и normalised_name не содержатся данные

In [24]:
# выбираем строки, в которых одновременно в колонках "Name" и "normalised_name" отсутствуют значения
missing_values = df[(df['Name'].isnull()) & (df['normalised_name'].isnull())]

if len(missing_values) > 0:
    print("Есть строки, в которых одновременно в колонках 'Name' и 'normalised_name' отсутствуют значения:")
    print(missing_values)
else:
    print("Нет строк, в которых одновременно в колонках 'Name' и 'normalised_name' отсутствуют значения")

Нет строк, в которых одновременно в колонках 'Name' и 'normalised_name' отсутствуют значения


### Заполним пропуски в normalised_name

In [25]:
# выборка индексов строк с пропущенными значениями в колонке normalised_name
null_indexes = df[df['normalised_name'].isnull()].index.tolist()

# создание списка значений из колонки Name по индексам из null_indexes
name_indexes = df.loc[null_indexes, 'Name'].tolist()

null_id_name = [[x, y] for x, y in zip(null_indexes, name_indexes)]
print('Количество пропущенных значений в normalised_name:', len(null_id_name))
print(null_id_name)

Количество пропущенных значений в normalised_name: 1993
[[4, 'Alumina'], [8, 'CH3SO4'], [11, 'DMF'], [13, 'CuSO4·5H2O'], [18, 'Ge25Sb10Bi5S60'], [21, 'Y2O3'], [27, 'Rib'], [29, 'Sapphire'], [31, 'Glycerol'], [34, 'Acrylate'], [39, 'CdSe'], [51, 'Acetonitrile'], [53, 'Spermidine'], [54, 'Toluene'], [63, 'RbBr'], [64, 'HfO2'], [66, 'DMC'], [67, 'PDMS'], [68, 'ChCL'], [77, 'Quartz'], [84, 'MIBK'], [85, 'Trimethylamine'], [90, 'MgO'], [94, 'cholesterol'], [100, 'NaFeSi2O6'], [101, 'CaF2'], [102, 'quartz'], [107, 'clenbuterol'], [111, 'Toluene'], [114, 'carbon'], [121, 'BSO'], [122, 'Phenyl'], [125, 'DAP'], [130, 'Stilbene'], [134, 'DMF'], [136, 'B40Bi60Sm1.0'], [137, 'ETBE'], [138, 'BiZnBo–'], [141, 'Propane'], [142, 'TAG'], [143, 'BFTZ'], [145, 'SiTi'], [148, 'azo'], [150, 'polystyrene'], [154, 'DMSO'], [155, 'Hexane'], [160, 'SnS2'], [161, 'CCl4'], [165, 'polycarbonate'], [167, 'silica'], [168, 'alcohol'], [169, 'Poloxamer 188'], [171, 'Glycerol'], [173, 'TiO2'], [174, 'oxygen'], [175, '

#### Пробуем PubChemPy

In [26]:
def get_smiles(compound_dict):
    cid = compound_dict['CID']
    smiles = None
#   пробуем найти смайлс по CID
    try:
        compound = pbp.Compound.from_cid(cid)
        smiles = compound.isomeric_smiles
    except pbp.PubChemHTTPError:
        pass
#   если не получилось, то пробуем по синонимам найти смайлс
    if not smiles:
        for syn in compound_dict['Synonym']:
            try:
                results = pbp.get_compounds(syn, 'name')
                for result in results:
                    if 'isomeric_smiles' in result:
                        smiles = result.isomeric_smiles
                        break
            except pbp.PubChemHTTPError:
                pass
            if smiles:
                break
    return smiles

In [27]:
for i in range(len(null_id_name)):
#   не ко всем соединениям найдутся CID и синонимы, у таких веществ будет пустой элемент в списке (длина = 0)    
    if len(pbp.get_synonyms(null_id_name[i][1], 'name')) != 0:
#   получаем словарь формата {'CID': 123, 'Synonym': ['abc', ..., 'cde']}
        compound_dict = pbp.get_synonyms(null_id_name[i][1], 'name')[0]
#       найдем смайлс по CID или синонимам
        smiles = get_smiles(compound_dict)
#       добавим smiles по номеру строки и названию колонки в df
        print(smiles, null_id_name[i][1], null_id_name[i][0])
        df.at[null_id_name[i][0], 'normalised_name'] = smiles
    else:
        continue

O=[Al]O[Al]=O Alumina 4
COS(=O)(=O)[O-] CH3SO4 8
CN(C)C=O DMF 11
C1[C@H]([C@H]([C@H](C(O1)O)O)O)O Rib 27
C(C(CO)O)O Glycerol 31
C=CC(=O)[O-] Acrylate 34
[Se-2].[Cd+2] CdSe 39
CC#N Acetonitrile 51
C(CCNCCCN)CN Spermidine 53
CC1=CC=CC=C1 Toluene 54
[Br-].[Rb+] RbBr 63
C[Si](C)(C)O[Si](C)(C)O[Si](C)(C)C PDMS 67
O=[Si]=O Quartz 77
CC(C)CC(=O)C MIBK 84
CN(C)C Trimethylamine 85
C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2CC=C4[C@@]3(CC[C@@H](C4)O)C)C cholesterol 94
[F-].[F-].[Ca+2] CaF2 101
O=[Si]=O quartz 102
CC(C)(C)NCC(C1=CC(=C(C(=C1)Cl)N)Cl)O clenbuterol 107
CC1=CC=CC=C1 Toluene 111
[C] carbon 114
CCCCS(=N)(=O)CCC(C(=O)O)N BSO 121
C1=CC=[C]C=C1 Phenyl 122
C1=CC=C(C=C1)/C=C/C2=CC=CC=C2 Stilbene 130
CN(C)C=O DMF 134
CCOC(C)(C)C ETBE 137
CCC Propane 141
C1[C@H]([C@@H]([C@@H](C(O1)(CO)O)O)O)O TAG 142
CC1=C(C=CC(=C1)C2=CC(=C(C=C2)N=NC3=C(C4=CC=CC=C4C(=C3)S(=O)(=O)[O-])O)C)N=NC5=C(C6=CC=CC=C6C(=C5)S(=O)(=O)[O-])O.[Na+].[Na+] azo 148
CS(=O)C DMSO 154
CCCCCC Hexane 155
C(Cl)(Cl)(Cl)Cl CC

In [28]:
# выборка индексов строк с пропущенными значениями в колонке normalised_name
null_indexes = df[df['normalised_name'].isnull()].index.tolist()

# создание списка значений из колонки Name по индексам из null_indexes
name_indexes = df.loc[null_indexes, 'Name'].tolist()

null_id_name = [[x, y] for x, y in zip(null_indexes, name_indexes)]
print('Количество пропущенных значений в normalised_name:', len(null_id_name))

Количество пропущенных значений в normalised_name: 606


In [29]:
# запринтим гидраты из датасета
for i in range(len(null_id_name)):
    if '·' in null_id_name[i][1] and 'H2O' in null_id_name[i][1]:
        print(null_id_name[i][1], null_id_name[i][0])

CuSO4·5H2O 13
CuSO4·H2O 3302
CuSO4·3H2O 3664
CuCl2·2H2O 3835


In [30]:
# гидраты в pubchempy записываются в таком формате abc.5H2O
contains_dot = []
for i in range(len(null_id_name)):
    if '·' in null_id_name[i][1] and 'H2O' in null_id_name[i][1]:
#       не все гидраты можно найти в таком формате
        if len(pbp.get_synonyms(null_id_name[i][1].replace('·', '.'), 'name')) != 0:
#           получаем словарь формата {'CID': 123, 'Synonym': ['abc', ..., 'cde']}
            compound_dict = pbp.get_synonyms(null_id_name[i][1].replace('·', '.'), 'name')[0]
#           найдем смайлс по CID или синонимам
            smiles = get_smiles(compound_dict)
#           добавим smiles по номеру строки и названию колонки в df
            print(smiles, null_id_name[i][1], null_id_name[i][0])
            df.at[null_id_name[i][0], 'normalised_name'] = smiles
        else:
            continue

O.O.O.O.O.[O-]S(=O)(=O)[O-].[Cu+2] CuSO4·5H2O 13
O.[O-]S(=O)(=O)[O-].[Cu+2] CuSO4·H2O 3302
O.O.Cl[Cu]Cl CuCl2·2H2O 3835


In [31]:
# выборка индексов строк с пропущенными значениями в колонке normalised_name
null_indexes = df[df['normalised_name'].isnull()].index.tolist()

# создание списка значений из колонки Name по индексам из null_indexes
name_indexes = df.loc[null_indexes, 'Name'].tolist()

null_id_name = [[x, y] for x, y in zip(null_indexes, name_indexes)]
print('Количество пропущенных значений в normalised_name:', len(null_id_name))

Количество пропущенных значений в normalised_name: 603


#### Пробуем PubChem Identifier Exchange Service

In [32]:
for i in range(len(null_id_name)):
#   отправляем запрос к PubChem Identifier Exchange Service
    response = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{null_id_name[i][1]}/property/CanonicalSMILES/json')
#   проверяем, успешно ли был получен ответ
    if response.status_code == 200:
#       извлекаем smiles и/или CID из JSON-ответа
        smiles = None
        try:
            smiles = response.json()['PropertyTable']['Properties'][0]['CanonicalSMILES']
        except:
            pass
        if not smiles:
            try:
                cid = response.json()['PropertyTable']['Properties'][0]['CID']
                compound = pbp.Compound.from_cid(cid)
                smiles = compound.isomeric_smiles
            except:
                pass
#       добавим smiles по номеру строки и названию колонки в df
        if 'smiles' and 'cid' in locals():
            print('Это CID:', cid, 'Это smiles:', smiles, null_id_name[i][1], null_id_name[i][0])
        elif 'smiles' in locals():
            print('Это smiles:', smiles, null_id_name[i][1], null_id_name[i][0])
        df.at[null_id_name[i][0], 'normalised_name'] = smiles
    else:
        continue

Это smiles: C1C(C(C(C(O1)O)O)O)O PEN 1249
Это smiles: *C methyl 3442


In [33]:
# выборка индексов строк с пропущенными значениями в колонке normalised_name
null_indexes = df[df['normalised_name'].isnull()].index.tolist()

# создание списка значений из колонки Name по индексам из null_indexes
name_indexes = df.loc[null_indexes, 'Name'].tolist()

null_id_name = [[x, y] for x, y in zip(null_indexes, name_indexes)]
print('Количество пропущенных значений в normalised_name:', len(null_id_name))

Количество пропущенных значений в normalised_name: 601


#### Пробуем ChEMBL

In [34]:
for i in range(len(null_id_name)):
    compound_name = null_id_name[i][1]
#   URL для получения информации из базы данных ChEMBL
    url = f'https://www.ebi.ac.uk/chembl/api/data/molecule?pref_name__iexact={compound_name}&format=json'
    response = requests.get(url)
    if response.ok:
        data = response.json()
        for mol in data['molecules']:
            smiles = None
            try:
                smiles = mol['molecule_structures']['canonical_smiles']
            except:
                pass
            if not smiles:
                try:
                    inchi_key = mol['molecule_structures']['standard_inchi_key']
                    compound = pbp.get_compounds(inchi_key, 'inchikey')[0]
                    smiles = compound.isomeric_smiles
                except:
                    pass
            print('Это smiles:', smiles, null_id_name[i][1], null_id_name[i][0])
            df.at[null_id_name[i][0], 'normalised_name'] = smiles    

Это smiles: O=[Si](O)O silicate 1521
Это smiles: O=[Si](O)O Silicate 1633
Это smiles: O=[Si](O)O Silicate 3008
Это smiles: O=[Si](O)O silicate 4888


In [35]:
# выборка индексов строк с пропущенными значениями в колонке normalised_name
null_indexes = df[df['normalised_name'].isnull()].index.tolist()

# создание списка значений из колонки Name по индексам из null_indexes
name_indexes = df.loc[null_indexes, 'Name'].tolist()

null_id_name = [[x, y] for x, y in zip(null_indexes, name_indexes)]
print('Количество пропущенных значений в normalised_name:', len(null_id_name))

Количество пропущенных значений в normalised_name: 597


#### Пробуем Cactus Chemical Identifier Resolver

In [37]:
def get_smiles_from_cir(compound_name):
#   отправляем запрос к CIR
    response = requests.get(f'https://cactus.nci.nih.gov/chemical/structure/{compound_name}/smiles')
#   извлекаем SMILES из ответа
    smiles = None
    if response.status_code == 200:
        smiles = response.content.decode().strip()
    return smiles

In [38]:
for i in range(len(null_id_name)):
    compound_name = null_id_name[i][1]
    smiles = get_smiles_from_cir(compound_name)
#   добавим smiles по номеру строки и названию колонки в df
    print(smiles, null_id_name[i][1], null_id_name[i][0])
    df.at[null_id_name[i][0], 'normalised_name'] = smiles

None Ge25Sb10Bi5S60 18
None Y2O3 21
O=[Al]O[Al]=O Sapphire 29
None HfO2 64
CC(O)(c1ccc(Cl)cc1)c2ccc(Cl)cc2 DMC 66
CCl ChCL 68
[OH-]|[Mg++](|[OH-])(|[OH-])(|[OH-])(|[OH-])|[OH-] MgO 90
None NaFeSi2O6 100
Nc1cccc(N)n1 DAP 125
None B40Bi60Sm1.0 136
None BiZnBo– 138
None BFTZ 143
None SiTi 145
C=Cc1ccccc1 polystyrene 150
None SnS2 160
CC(C)(c1ccc(O)cc1)c2ccc(O)cc2 polycarbonate 165
CC(C)[C@H](CO)Nc1nc(Nc2ccc(C(O)=O)c(Cl)c2)c3ncn(C(C)C)c3n1 PVB 175
CCl ChCL 176
ClC=C PVC 198
None Si–H(N2Si) 203
None RbTi0.971Yb0.015Nb0.014OPO4 211
None TeEr1 214
None Si3N4 224
None BSMLN 233
None As2S3 240
None SiO2–Nb2O5 245
None CdSnAs2 248
None Tb3Ga5O12 250
None BiFeO3 256
None SU-8 261
None TiO2-SiO2 262
None Y2O3 277
CC=C polypropylene 280
O=[Al]O[Al]=O Sapphire 287
None As30Se70 302
O=[Al]O[Al]=O Sapphire 315
C=CN1CCCC1=O Poly(N-vinylpyrrolidone) 317
None Ca2Al2SiO7 327
None Mg3Al2Si3O12(P) 333
None Sb2O3–GeO2 342
None B10P20T70 346
OCCO.OCCCCO.OC(=O)CCCCC(O)=O.O=C=Nc1ccc(Cc2ccc(cc2)N=C=O)cc1 polyure

In [39]:
# выборка индексов строк с пропущенными значениями в колонке normalised_name
null_indexes = df[df['normalised_name'].isnull()].index.tolist()

# создание списка значений из колонки Name по индексам из null_indexes
name_indexes = df.loc[null_indexes, 'Name'].tolist()

null_id_name = [[x, y] for x, y in zip(null_indexes, name_indexes)]
print('Количество пропущенных значений в normalised_name:', len(null_id_name))

Количество пропущенных значений в normalised_name: 419


#### Пробуем Chembl_webresource_client

In [46]:
for i in range(len(null_id_name)):
    molecule = new_client.molecule
    compound_name = null_id_name[i][1]
    smiles = None
    try:
        compound = molecule.search(compound_name)[0]
        smiles = compound['molecule_structures']['canonical_smiles']
    except:
        pass
#   добавим smiles по номеру строки и названию колонки в df
    print(smiles, null_id_name[i][1], null_id_name[i][0])
    df.at[null_id_name[i][0], 'normalised_name'] = smiles

None Ge25Sb10Bi5S60 18
None Y2O3 21
None HfO2 64
None NaFeSi2O6 100
None B40Bi60Sm1.0 136
None BiZnBo– 138
None BFTZ 143
None SiTi 145
None SnS2 160
CC[C@H](C)[C@H](N)C(=O)N[C@H]1CSSC[C@@H]2NC(=O)[C@H](C)NC(=O)[C@@H]3CCCN3C(=O)[C@H](CC(N)=O)NC(=O)[C@H](CSSC[C@@H](C(N)=O)NC(=O)[C@H](CO)NC(=O)[C@H](Cc3ccc(O)cc3)NC(=O)[C@H](CCCCN)NC(=O)[C@@H]3CCCN3C(=O)CNC2=O)NC1=O.COc1ccc2cc1Oc1cc3c(cc1OC)CC[N+](C)(C)[C@H]3Cc1ccc(cc1)Oc1c(OC)c(OC)cc3c1[C@@H](C2)[N+](C)(C)CC3.[I-].[I-] Si–H(N2Si) 203
None RbTi0.971Yb0.015Nb0.014OPO4 211
None TeEr1 214
None Si3N4 224
None BSMLN 233
None As2S3 240
None SiO2–Nb2O5 245
None CdSnAs2 248
None Tb3Ga5O12 250
None BiFeO3 256
None SU-8 261
None TiO2-SiO2 262
None Y2O3 277
None As30Se70 302
None Ca2Al2SiO7 327
C[C@H](O)CCCCn1c(=O)c2c(ncn2C)n(C)c1=O Mg3Al2Si3O12(P) 333
None Sb2O3–GeO2 342
None B10P20T70 346
None Bi2O3.xSiO20.5Sm2O3 359
None AgGaSe2 382
None ZrON 399
None BMST 406
CCCCOCCOCCOCc1cc2c(cc1CCC)OCO2 Li2O–GeO2–PbO–Nd2O3 432
None Ta2O5 435
None Y2O3 445
None

In [47]:
# выборка индексов строк с пропущенными значениями в колонке normalised_name
null_indexes = df[df['normalised_name'].isnull()].index.tolist()

# создание списка значений из колонки Name по индексам из null_indexes
name_indexes = df.loc[null_indexes, 'Name'].tolist()

null_id_name = [[x, y] for x, y in zip(null_indexes, name_indexes)]
print('Количество пропущенных значений в normalised_name:', len(null_id_name))

Количество пропущенных значений в normalised_name: 334


#### Пробуем ChemSpider

In [79]:
# Создание объекта ChemSpider с использованием вашего API-ключа
cs = chemspipy.ChemSpider('FrpLnPw0HSVI2v6eyOvPZIM5Aho6YhgL')
for i in range(len(null_id_name)):
    smiles = None
#   Поиск химического соединения по его названию
    result = cs.search(null_id_name[i][1])
    if len(result) != 0:
        smiles = result[0].smiles
    else:
        continue
#   добавим smiles по номеру строки и названию колонки в df
    print(smiles, null_id_name[i][1], null_id_name[i][0])
    df.at[null_id_name[i][0], 'normalised_name'] = smiles

[F-].[F-].[F-].[F-].[Na+].[Y+3] NaYF4 595
CCCCCCc1cc(sc1Br)Br P3HT 615
N1=[Al]N2[Al]=N[Al]3N=[Al]N4[Si]2(O3)O[Al]1N=[Al]4 SiAlON 1012
[Cd]=[Te] CdTe 1383
[As](=[Se])[Se][As]=[Se] As2Se3 1832
[Cd]=[Te] CdTe 2177
O([Cu])[Cu] Cu2O 2322
[Ga]#[As] GaAs 2952
O([Cu])[Cu] Cu2O 3082
C1=CC=C2C(=C1)C(=O)OC23C4=C(C=C(C=C4)[O-])OC5=C3C=CC(=C5)[O-].[Na+].[Na+] NaFl 3180
[Ga]#[As] GaAs 3203
[Cu+].[Br-] CuBr 3382
O([Cu])[Cu] Cu2O 3743
[Ga]#[As] GaAs 3810
N1=[Al]N2[Al]=N[Al]3N=[Al]N4[Si]2(O3)O[Al]1N=[Al]4 SiAlON 4667
O([Cu])[Cu] Cu2O 4827
CCCCCCc1cc(sc1Br)Br P3HT 4920


In [80]:
# выборка индексов строк с пропущенными значениями в колонке normalised_name
null_indexes = df[df['normalised_name'].isnull()].index.tolist()

# создание списка значений из колонки Name по индексам из null_indexes
name_indexes = df.loc[null_indexes, 'Name'].tolist()

null_id_name = [[x, y] for x, y in zip(null_indexes, name_indexes)]
print('Количество пропущенных значений в normalised_name:', len(null_id_name))

Количество пропущенных значений в normalised_name: 317


In [81]:
# Сохраним промежуточный вариант df
df.to_csv('6.csv', index=False)